In [11]:
import numpy as np
from pathlib import Path
import cv2
import pandas as pd
from skimage import measure, filters
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
import tqdm 

import SimpleITK as sitk
import itkwidgets 
import napari

from napari.utils.colormaps import colormap_utils as cu

In [3]:
mask_list = np.load(Path("temp/mask_list.npy"))
mask1=mask_list[0]
mask1=np.where(mask1==255, 0, 1)
print(mask1.shape)

(1532, 512, 512)


In [3]:
# previous_area=500
# enchainement_T_F_area=[] #true if the next is smaller
# previous_ML=100
# enchainement_major_length=[] #true if the next is smaller
# for im_in in mask1[-1:]:
#     im_floodfill = im_in.copy()
#     h, w = im_in.shape[:2]
#     mask_new = np.zeros((h+2, w+2), np.uint8)
#     cv2.floodFill(im_floodfill, mask_new, (0,0), 0)
    
#     sum_im_in=np.sum(im_floodfill)
#     if sum_im_in < previous_area:
#         enchainement_T_F_area.append(True)
#     else: 
#         enchainement_T_F_area.append(False)
#     previous_area=sum_im_in
    
#     properties_to_test=["axis_major_length"] 
#     props = measure.regionprops_table(im_floodfill, im_in, properties=properties_to_test)
#     major_Lenght=(props["axis_major_length"][0])

#     if major_Lenght < previous_ML:
#         enchainement_major_length.append(True)
#     else: 
#         enchainement_major_length.append(False)
#     previous_ML=major_Lenght
        
#     fig, ax = plt.subplots(nrows = 1, ncols=2, figsize=(15,6))
#     ax[0].imshow(im_in[270:310, 220:300])
#     ax[1].imshow(im_floodfill[270:310, 220:300])
#     plt.show()
# import collections
# frequency = collections.Counter(enchainement_T_F_area)
# print(frequency)
# frequency2 = collections.Counter(enchainement_major_length)
# print(frequency2)


In [12]:
def fit_ellipses(mask, rangemin, rangemax):
    ''' 
    Le but est de récuperer les paramètres d'une liste ellipse en fonction d'une liste de masques fournis par le network construction
    Args :
        mask (numpy array) : masque des artères
        rangemin (int): début des coupes sélectionnées
        rangemax (int): fin des coupes sélectionnées
    Return:
        list_params_ellispses (list[list[double]]): paramètres de chaque ellipse
    '''
    list_params_ellipses=[]
    for img in tqdm.tqdm(mask[rangemin:rangemax]):
        list_points_ellipse=[]
        for j in range(img.shape[0]):
            for i in range(img.shape[1]):
                if img[j][i] ==0:
                    list_points_ellipse.append((i,j))
        array_points_ellipse=np.array(list_points_ellipse)

        ell = measure.EllipseModel()
        ell.estimate(array_points_ellipse)
        list_params_ellipses.append(ell.params)
    return list_params_ellipses

In [13]:
def create_3d_ellipses(list_params_ellipses):
    '''
    Crée le numpy array représentant les images des ellipses
    Args:
        list_params_ellispses (list[list[double]]): paramètres de chaque ellipse
    Return:
        ellipses_3d (numpy array) : 3d images of the ellipses
    '''
    list_2D_ellipses = []
    for params in list_params_ellipses:
        xc, yc, a, b, theta = params
        t = np.linspace(0, 2*np.pi, 100)
        X = xc + a*np.cos(theta)*np.cos(t) - b*np.sin(theta)*np.sin(t)
        Y = yc + a*np.sin(theta)*np.cos(t) + b*np.cos(theta)*np.sin(t)
        built_ellipse = np.zeros((512,512))
        for i in range(len(X)):
            built_ellipse[int(Y[i])][int(X[i])] = 1

        list_2D_ellipses.append(built_ellipse)
    ellipses_3D = np.array(list_2D_ellipses)
    return ellipses_3D

In [14]:
rangemin=mask1.shape[0]-300
rangemax=mask1.shape[0]

list_params = fit_ellipses(mask1, rangemin,rangemax)
ellipses_3d = create_3d_ellipses(list_params)

100%|██████████| 300/300 [00:47<00:00,  6.25it/s]


In [16]:
# to view in napari
image_array = np.load(Path("temp/preprocessed_image.npy"))
image_originale=image_array[rangemin:rangemax]
mask_lucas=mask1[rangemin:rangemax]

viewer = napari.Viewer()

colorindex = 0
image_layer = viewer.add_image(image_originale)
image_layer = viewer.add_image(ellipses_3d,colormap=list(cu.AVAILABLE_COLORMAPS.keys())[colorindex],blending="additive")
image_layer = viewer.add_image(mask_lucas,colormap=list(cu.AVAILABLE_COLORMAPS.keys())[colorindex],blending="additive")
napari.run()

tests pour smooth la surface

In [18]:
# test algo à la main
# list_params_ellipses : xc, yc, a, b, theta
# [] : 1 ellipse [][] : le param
import copy

count = 0
seuil = [3,3,70,70,100]
new_list_params_ellipses = copy.deepcopy(list_params_ellipses)
#new_list_params_ellipses.append(list_params_ellipses[0])
# 1st step : voir si on a des ellipses qui déconnent
for i in range(len(new_list_params_ellipses)-1): # pour toutes les ellipses
    for j in range(len(new_list_params_ellipses[0])): # pour chacun des paramètres
        param_avant = new_list_params_ellipses[i][j]
        param = new_list_params_ellipses[i+1][j]
        if (param-param_avant)*100/param_avant > seuil[j]:
            count +=1
            new_list_params_ellipses[i+1][j] = param_avant

print(count)

234


In [19]:
# create the new ellipses using parameters
new_ellipses = create_3d_ellipses(new_list_params_ellipses)

# look at the result
viewer = napari.Viewer()

colorindex = 0
image_layer = viewer.add_image(ellipses_3d)
#image_layer = viewer.add_image(ellipse_to_see,colormap=list(cu.AVAILABLE_COLORMAPS.keys())[colorindex],blending="additive")
image_layer = viewer.add_image(new_ellipses,colormap=list(cu.AVAILABLE_COLORMAPS.keys())[colorindex],blending="additive")
napari.run()

In [32]:
list_xc = []
list_yc = []
for i in range(len(new_list_params_ellipses)):
    xc, yc, a, b, theta = new_list_params_ellipses[i]
    list_xc.append(xc)
    list_yc.append(yc)

print(np.mean(list_xc))
print(np.std((list_xc)))
print(np.mean(list_yc))
print(np.std(list_yc))

257.11504018926564
6.290653473854987
240.56234515703187
27.664639993439074
